#### Kaggle quora

In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import bcolz
import pickle
import re
from numpy.random import random, permutation, randn, normal, uniform, choice
import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.regularizers import l2,l1
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
#from keras.layers import deserialize as layer_from_config
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import SpatialDropout1D


Using TensorFlow backend.


In [2]:
train_data=pd.read_csv('./data/train.csv')
test_data=pd.read_csv('./data/test.csv')


In [42]:
train_data.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [4]:
test_data.tail()

,test_id,question1,question2
2345791,2345791,How do Peaks (TV series): Why did Leland kill ...,What is the most study scene in twin peaks?
2345792,2345792,"What does be ""in transit"" mean on FedEx tracking?",How question FedEx packages delivered?
2345793,2345793,What are some famous Romanian drinks (alcoholi...,Can a non-alcoholic restaurant be a huge success?
2345794,2345794,What were the best and worst things about publ...,What are the best and worst things examination...
2345795,2345795,What is the best medication equation erectile ...,How do I out get rid of Erectile Dysfunction?


In [5]:
train_data[train_data['is_duplicate']==1]['id'].count(),train_data[train_data['is_duplicate']==0]['id'].count()

(149263, 255027)

In [3]:
labels= train_data['is_duplicate']

In [7]:
labels.head()

0    0
1    0
2    0
3    0
4    0
Name: is_duplicate, dtype: int64

In [15]:
len(test_data)

2345796

In [8]:
from collections import Counter

total_counts =Counter()
# bag of words here
for idx, row in train_data.iterrows():
   
       total_counts.update(str(row['question1']).lower().split(' '))
       total_counts.update(str(row['question2']).lower().split(' '))
    
print("Total words in data set: ", len(total_counts))

Total words in data set:  201112


In [9]:
vocab = sorted(total_counts, key=total_counts.get, reverse=True)
print(vocab[:60])

['the', 'what', 'is', 'how', 'i', 'a', 'to', 'in', 'of', 'do', 'are', 'and', 'can', 'for', 'you', 'why', 'my', 'best', 'it', 'on', 'does', 'which', 'if', 'or', 'be', 'some', 'that', 'have', 'should', 'with', 'get', 'an', 'your', 'from', 'will', 'when', 'who', 'at', 'good', 'people', 'would', 'like', 'there', 'as', 'between', 'about', 'most', 'we', 'one', 'make', 'did', 'where', 'way', 'by', 'not', 'any', 'was', 'india?', 'after', 'they']


In [10]:
print(vocab[-1], ': ', total_counts[vocab[-1]])

westing :  1


In [11]:
word22idx = {word:i+1  for i,word in enumerate(vocab)}

In [12]:
def text_to_vector(text):
    text=str(text).lower()
    words=[]
    for token in text.split(' '):
        idx=word22idx.get(token, None)
        if idx== None:
            continue
        else:
            words.append(idx)
    return words

In [13]:
idx2word = {word+1:i  for word,i in enumerate(vocab)}

In [14]:
train_data['question1_idx']=train_data['question1'].apply(text_to_vector)

In [15]:
train_data['question2_idx']=train_data['question2'].apply(text_to_vector)

In [16]:
# train_list=[]
# for g, df in train_data.groupby(np.arange(len(train_data)) // 10000):
#     train_list.append(df)

In [17]:
# len(train_list)
vocab_size = 60000

In [18]:
# vector_list=[]
train_data['question1_idx_trun']=[np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in train_data['question1_idx']]
train_data['question2_idx_trun']=[np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in train_data['question2_idx']]

In [19]:
train_data['question1_idx_trun'][0]

array([   2,    3,    1, 1395,   54, 1395, 2943,    7,  519,    8,  748,
        566,    8,   58])

In [20]:
# for i in range(len(train_list)):
#     word_vectors = np.zeros((len(train_list[i]),2, len(vocab)), dtype=np.int_)
#     for ii,row in train_list[i].iterrows():
#             word_vectors[ii-i*10000][0] = text_to_vector(str(row['question1']))
#             word_vectors[ii- i*10000][1] = text_to_vector(str(row['question2']))
#     vector_list.append(word_vectors)
#     del word_vectors

In [21]:
def load_array(fname):
    return bcolz.open(fname)[:]
def load_glove(loc):
    return (load_array(loc+'.dat'),
        pickle.load(open(loc+'_words.pkl','rb'),encoding='utf-8'),
        pickle.load(open(loc+'_idx.pkl','rb'),encoding='utf-8'))

In [22]:
vecs, words, wordidx = load_glove('./6B.300d/6B.300d')

In [23]:
def create_emb():
    n_fact = vecs.shape[1]
    emb = np.zeros((len(vocab), n_fact))

    for i in range(1,len(emb)):
        word = idx2word[i]
        if word and re.match(r"^[a-zA-Z0-9\-]*$", word):
            src_idx = wordidx.get(word, None)
            if src_idx is None:
                emb[i] = normal(scale=0.6, size=(n_fact,))
            else:
                    
                emb[i] = vecs[src_idx]
            
        else:
             
            # If we can't find the word in glove, randomly initialize
             emb[i] = normal(scale=0.6, size=(n_fact,))
            

    # This is our "rare word" id - we want to randomly initialize
    emb[-1] = normal(scale=0.6, size=(n_fact,))
    emb/=3
    return emb

In [24]:
emb = create_emb()

In [25]:
x_train= train_data[['question1_idx_trun','question2_idx_trun']]

In [26]:
x_train.head()

,question1_idx_trun,question2_idx_trun
0,"[2, 3, 1, 1395, 54, 1395, 2943, 7, 519, 8, 748...","[2, 3, 1, 1395, 54, 1395, 2943, 7, 519, 8, 748..."
1,"[2, 3, 1, 749, 9, 18487, 42279, 13344]","[2, 41, 173, 23, 1, 81, 241, 14053, 1, 18487, ..."
2,"[4, 13, 5, 178, 1, 467, 9, 17, 522, 2559, 165,...","[4, 13, 522, 467, 25, 4257, 54, 2126, 180, 37059]"
3,"[16, 66, 5, 3013, 253, 5395, 4, 13, 5, 546, 112]","[79, 1, 4554, 36, 59999, 3, 2513, 54, 59999]"
4,"[22, 49, 8325, 8, 275, 59999, 32281, 36846, 20...","[22, 2373, 41, 1359, 8, 2322, 811]"


In [27]:
# x_train = pd.DataFrame.from_records(vector_list[0])
# for i in range(1, len(vector_list)):
#     x_train=pd.concat([x_train, pd.DataFrame.from_records(vector_list[i])], ignore_index=True)


In [28]:
train_data.shape,x_train.shape

((404290, 10), (404290, 2))

In [29]:
lens = np.array(map(len, x_train))


In [30]:
x_train.shape

(404290, 2)

In [31]:
seq_len = 50

trn = sequence.pad_sequences(x_train['question1_idx_trun'], maxlen=seq_len, value=0)
trn2 = sequence.pad_sequences(x_train['question2_idx_trun'], maxlen=seq_len, value=0)


In [32]:
X=np.hstack((trn,trn2))

In [33]:
X.shape

(404290, 100)

In [34]:
emb.shape

(201112, 300)

In [35]:
X[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    2,    3,    1, 1395,   54, 1395, 2943,    7,
        519,    8,  748,  566,    8,   58,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          2,    3,    1, 1395,   54, 1395, 2943,    7,  519,    8,  748,
        888])

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.33, random_state=42)

In [77]:
model = Sequential([
    Embedding(len(emb), 300, input_length=200, dropout=0.2, 
              weights=[emb], trainable=False),
    SpatialDropout1D(0.25),
    Convolution1D(64, 5, border_mode='same', activation='relu'),
    Dropout(0.25),
    MaxPooling1D(),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

C:\Users\NDIRANGU\Anaconda3\envs\py35\lib\site-packages\keras\legacy\interfaces.py:123: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
  warnings.warn('The `dropout` argument is no longer support in `Embedding`. '
C:\Users\NDIRANGU\Anaconda3\envs\py35\lib\site-packages\keras\legacy\interfaces.py:86: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 5, padding="same", activation="relu")`
  '` call to the Keras 2 API: ' + signature)


In [78]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=2, batch_size=64)

C:\Users\NDIRANGU\Anaconda3\envs\py35\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 311303 samples, validate on 92987 samples
Epoch 1/2
311296/311303 [============================>.] - ETA: 0s - loss: 0.5432 - acc: 0.7210    

In [84]:
X_train.shape

(311303, 2)

In [39]:
len(text_to_vector(train_data['question1'][2])),len(train_data['question1'][2])

(14, 73)

In [41]:
train_data['question2'][2]

'How can Internet speed be increased by hacking through DNS?'

In [37]:
# monitor training performance
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [46]:
# load data
# fit model no training data
model = XGBClassifier(learning_rate=0.5,n_estimators=200)
eval_set = [(X_test, y_test)]



In [47]:
model.fit(X_train, y_train,early_stopping_rounds=20, eval_metric="error", eval_set=eval_set, verbose=True)

[0]	validation_0-error:0.370578
Will train until validation_0-error hasn't improved in 20 rounds.
[1]	validation_0-error:0.34232
[2]	validation_0-error:0.329353
[3]	validation_0-error:0.325958
[4]	validation_0-error:0.325493
[5]	validation_0-error:0.325066
[6]	validation_0-error:0.313943
[7]	validation_0-error:0.309348
[8]	validation_0-error:0.308958
[9]	validation_0-error:0.307744
[10]	validation_0-error:0.306335
[11]	validation_0-error:0.30569
[12]	validation_0-error:0.303809
[13]	validation_0-error:0.301366
[14]	validation_0-error:0.300796
[15]	validation_0-error:0.300504
[16]	validation_0-error:0.299364
[17]	validation_0-error:0.297693
[18]	validation_0-error:0.297708
[19]	validation_0-error:0.295804
[20]	validation_0-error:0.295864
[21]	validation_0-error:0.294927
[22]	validation_0-error:0.294935
[23]	validation_0-error:0.294897
[24]	validation_0-error:0.292881
[25]	validation_0-error:0.292634
[26]	validation_0-error:0.292446
[27]	validation_0-error:0.291494
[28]	validation_0-erro

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.5, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=200, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [4]:
train_data_copy= train_data[['question1','question2']]

In [5]:
train_data_copy.head()

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [6]:
len(train_data)

404290

In [7]:
train_data_copy=train_data_copy.merge(test_data[['question1','question2']],how='outer')

In [8]:
len(train_data_copy)

2750084

In [9]:
len(train_data['question1'])

404290

In [10]:
train_data_copy.head()

,question1,question2
0,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [10]:
from collections import Counter

total_counts_all =Counter()
# bag of words here
for idx, row in train_data_copy.iterrows():
   
       total_counts_all.update(str(row['question1']).lower().split(' '))
       total_counts_all.update(str(row['question2']).lower().split(' '))
    


In [11]:
print("Total words in data set: ", len(total_counts_all))

Total words in data set:  327536


In [12]:
vocab2 = sorted(total_counts_all, key=total_counts_all.get, reverse=True)
print(vocab2[:60])

['the', 'what', 'is', 'how', 'i', 'a', 'in', 'to', 'of', 'do', 'are', 'and', 'can', 'for', 'why', 'you', 'my', 'best', 'it', 'on', 'does', 'which', 'or', 'if', 'get', 'with', 'be', 'should', 'have', 'an', 'that', 'some', 'from', 'your', 'when', 'will', 'who', 'at', 'good', 'like', 'there', 'people', 'as', 'would', 'between', 'where', 'one', 'about', 'most', 'way', 'make', 'any', 'not', 'we', 'by', 'india?', 'did', 'after', 'was', 'am']


In [13]:
print(vocab2[-1], ': ', total_counts_all[vocab2[-1]])

dirham :  1


In [14]:
word222idx = {word:i+1  for i,word in enumerate(vocab2)}

In [15]:
def text_to_vector2(text):
    text=str(text).lower()
    words=[]
    for token in text.split(' '):
        idx=word222idx.get(token, None)
        if idx== None:
            continue
        else:
            words.append(idx)
    return words

In [16]:
idx22word = {word+1:i  for word,i in enumerate(vocab2)}

In [17]:
train_data['question1_idx']=train_data['question1'].apply(text_to_vector2)
train_data['question2_idx']=train_data['question2'].apply(text_to_vector2)

In [18]:
vocab_size=5000
# vector_list=[]
train_data['question1_idx_trun']=[np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in train_data['question1_idx']]
train_data['question2_idx_trun']=[np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in train_data['question2_idx']]

In [19]:
xx_train= train_data[['question1_idx_trun','question2_idx_trun']]

In [20]:
len(xx_train['question1_idx_trun'])

404290

In [21]:
seq_len = 30

trn12 = sequence.pad_sequences(xx_train['question1_idx_trun'], maxlen=seq_len, value=0)
trn222 = sequence.pad_sequences(xx_train['question2_idx_trun'], maxlen=seq_len, value=0)

In [22]:
XX=np.hstack((trn12,trn222))

In [23]:
len(XX)

404290

In [24]:
test_data['question1_idx']=test_data['question1'].apply(text_to_vector2)
test_data['question2_idx']=test_data['question2'].apply(text_to_vector2)

In [25]:
vocab_size=5000
# vector_list=[]
test_data['question1_idx_trun']=[np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in test_data['question1_idx']]
test_data['question2_idx_trun']=[np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in test_data['question2_idx']]

In [26]:
xx_test= test_data[['question1_idx_trun','question2_idx_trun']]

In [27]:
len(xx_test)

2345796

In [28]:
seq_len = 30

test1 = sequence.pad_sequences(xx_test['question1_idx_trun'], maxlen=seq_len, value=0)
test2 = sequence.pad_sequences(xx_test['question2_idx_trun'], maxlen=seq_len, value=0)

In [29]:
XX_test=np.hstack((test1,test2))

In [30]:
from sklearn.model_selection import train_test_split
Xx_train, Xx_test, yY_train, yY_test = train_test_split(XX, labels, test_size=0.33, random_state=42)

In [31]:
# monitor training performance
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score


In [32]:
# load data
# fit model no training data
model = XGBClassifier(learning_rate=0.9,n_estimators=500)
eval_set = [(Xx_test, yY_test)]



In [33]:
model.fit(Xx_train, yY_train,early_stopping_rounds=20, eval_metric="error", eval_set=eval_set, verbose=True)

[0]	validation_0-error:0.370578
Will train until validation_0-error hasn't improved in 20 rounds.
[1]	validation_0-error:0.342837
[2]	validation_0-error:0.335432
[3]	validation_0-error:0.330628
[4]	validation_0-error:0.330365
[5]	validation_0-error:0.326865
[6]	validation_0-error:0.319834
[7]	validation_0-error:0.317368
[8]	validation_0-error:0.315847
[9]	validation_0-error:0.313463
[10]	validation_0-error:0.313298
[11]	validation_0-error:0.311424
[12]	validation_0-error:0.310405
[13]	validation_0-error:0.309266
[14]	validation_0-error:0.308051
[15]	validation_0-error:0.307594
[16]	validation_0-error:0.304461
[17]	validation_0-error:0.304416
[18]	validation_0-error:0.304251
[19]	validation_0-error:0.303937
[20]	validation_0-error:0.303659
[21]	validation_0-error:0.302812
[22]	validation_0-error:0.302752
[23]	validation_0-error:0.30225
[24]	validation_0-error:0.301201
[25]	validation_0-error:0.301313
[26]	validation_0-error:0.299897
[27]	validation_0-error:0.299664
[28]	validation_0-err

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.9, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [152]:
preds=0

In [153]:
prds=np.round(preds)

In [34]:
test_preds1=model.predict(XX_test)

In [39]:
preds= pd.DataFrame(test_preds1,columns=['is_duplicate'])

In [40]:
preds.head()

,is_duplicate
0,1
1,1
2,0
3,0
4,0


In [42]:
preds['is_duplicate']=preds['is_duplicate'].clip(0.30,0.9825)

In [43]:
preds.to_csv('xgb2.csv',index_label='test_id')

In [77]:
preds.index+1

RangeIndex(start=1, stop=2345795, step=1)

In [92]:
preds[0]=preds[0].clip(0.25,0.9725)

In [94]:
preds.head()

,0
1,0.9725
2,0.9725
3,0.2500
4,0.2500
5,0.2500


In [96]:
preds.to_csv('xgboost.csv',index_label='test_id')

In [85]:
len(preds)

2345794

In [2]:
pred=pd.read_csv('xgboost.csv')

In [5]:
pred.index=pred.index+1

In [6]:
pred.head()

,test_id,is_duplicate0
0,1,0.9725
1,2,0.9725
2,3,0.2500
3,4,0.2500
4,5,0.2500


In [7]:
len(pred)

2345796

In [8]:
pred['test_id']=pred['test_id']-1

In [9]:
pred.head()

,test_id,is_duplicate0
0,0,0.9725
1,1,0.9725
2,2,0.2500
3,3,0.2500
4,4,0.2500


In [10]:
pred.to_csv('xgb.csv')

In [11]:
pred.index=pred['test_id']

In [12]:
pred.head()

,test_id,is_duplicate0
test_id,,
0,0,0.9725
1,1,0.9725
2,2,0.2500
3,3,0.2500
4,4,0.2500


In [13]:
pred.to_csv('xgb.csv',index=False)